In [2]:
import sqlite3
import pandas as pd

DB_NAME = "election_results.db"

def get_state_winner_for_year(state_name, year):
    """
    Find the winner for a given state and year.
    For 2000-2020, data comes from historical_election_results.
    For 2024, data comes from the 'counties' and 'candidates' tables.
    """
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    if year < 2024:
        # Historical data query
        # Sum votes by candidate for given state and year, then order by votes desc
        query = """
            SELECT candidate, party, SUM(votes_candidate) as total_votes
            FROM historical_election_results
            WHERE state = ?
              AND year = ?
            GROUP BY candidate, party
            ORDER BY total_votes DESC
            LIMIT 1;
        """
        cursor.execute(query, (state_name, year))
        result = cursor.fetchone()
        if result:
            winner_candidate, winner_party, winner_votes = result
            return {"year": year, "state": state_name, 
                    "winner_candidate": winner_candidate, 
                    "winner_party": winner_party, 
                    "total_votes": winner_votes}
        else:
            return {"error": f"No data found for {state_name} in {year}."}
    else:
        # For 2024 data, we query counties & candidates tables
        query = """
            SELECT ca.name, ca.party, SUM(ca.votes) as total_votes
            FROM counties c
            JOIN candidates ca ON c.id = ca.county_id
            WHERE c.state = ?
            GROUP BY ca.name, ca.party
            ORDER BY total_votes DESC
            LIMIT 1;
        """
        cursor.execute(query, (state_name,))
        result = cursor.fetchone()
        if result:
            winner_candidate, winner_party, winner_votes = result
            return {"year": year, "state": state_name,
                    "winner_candidate": winner_candidate,
                    "winner_party": winner_party,
                    "total_votes": winner_votes}
        else:
            return {"error": f"No data found for {state_name} in {year}."}
    conn.close()


def get_unemployment_rate_by_fips_and_year(fips_code, year):
    """
    Get the unemployment rate for a given county fips_code and year.
    """
    conn = sqlite3.connect(DB_NAME)
    query = """
        SELECT unemp_rate 
        FROM unemployment_data
        WHERE fips_code = ?
          AND year = ?
        LIMIT 1;
    """
    df = pd.read_sql_query(query, conn, params=(fips_code, year))
    conn.close()

    if not df.empty:
        return {"fips_code": fips_code, "year": year, "unemp_rate": float(df['unemp_rate'].iloc[0])}
    else:
        return {"error": f"No unemployment data found for FIPS {fips_code} in {year}."}


def get_education_rates_by_fips(fips_code):
    """
    Get the available education data (HS diploma and Bachelor's+) for a given fips_code.
    The table includes data for 1990, 2000, 2008-12, and 2018-22.
    We'll return all four sets of columns.
    """
    conn = sqlite3.connect(DB_NAME)
    query = """
        SELECT bach_1990, bach_2000, bach_2008_12, bach_2018_22,
               hs_1990, hs_2000, hs_2008_12, hs_2018_22
        FROM education_data
        WHERE fips_code = ?
        LIMIT 1;
    """
    df = pd.read_sql_query(query, conn, params=(fips_code,))
    conn.close()

    if not df.empty:
        return {
            "fips_code": fips_code,
            "bach_1990": df['bach_1990'].iloc[0],
            "bach_2000": df['bach_2000'].iloc[0],
            "bach_2008_12": df['bach_2008_12'].iloc[0],
            "bach_2018_22": df['bach_2018_22'].iloc[0],
            "hs_1990": df['hs_1990'].iloc[0],
            "hs_2000": df['hs_2000'].iloc[0],
            "hs_2008_12": df['hs_2008_12'].iloc[0],
            "hs_2018_22": df['hs_2018_22'].iloc[0]
        }
    else:
        return {"error": f"No education data found for FIPS {fips_code}."}


def get_population_for_fips_and_year(fips_code, year):
    """
    Get the population estimate for a given county fips_code and year.
    """
    conn = sqlite3.connect(DB_NAME)
    query = """
        SELECT population_est
        FROM population_data
        WHERE fips_code = ?
          AND year = ?
        LIMIT 1;
    """
    df = pd.read_sql_query(query, conn, params=(fips_code, year))
    conn.close()

    if not df.empty:
        return {"fips_code": fips_code, "year": year, "population_est": int(df['population_est'].iloc[0])}
    else:
        return {"error": f"No population data found for FIPS {fips_code} in {year}."}


# Example usage:
print(get_state_winner_for_year("Alabama", 2020))
print(get_state_winner_for_year("Alabama", 2024))
print(get_unemployment_rate_by_fips_and_year(1001, 2010))
print(get_education_rates_by_fips(1001))
print(get_population_for_fips_and_year(1001, 2020))


{'error': 'No data found for Alabama in 2020.'}
{'error': 'No data found for Alabama in 2024.'}
{'fips_code': 1001, 'year': 2010, 'unemp_rate': 8.8}
{'fips_code': 1001, 'bach_1990': np.float64(14.5), 'bach_2000': np.float64(18.0), 'bach_2008_12': np.float64(21.707830639654), 'bach_2018_22': np.float64(29.558574698915095), 'hs_1990': np.float64(32.0), 'hs_2000': np.float64(33.8), 'hs_2008_12': np.float64(33.7867061233781), 'hs_2018_22': np.float64(31.146113267642082)}
{'fips_code': 1001, 'year': 2020, 'population_est': 56145}
